# Create a Arbitrage Betting Strategy

#### Install PolyMarket API library

In [1]:
!pip install py-clob-client

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for eth-account>=0.13.0 from https://files.pythonhosted.org/packages/46/18/088fb250018cbe665bc2111974301b2d59f294a565aff7564c4df6878da2/eth_account-0.13.7-py3-none-any.whl.metadata
  Obtaining dependency information for eth-utils>=4.1.1 from https://files.pythonhosted.org/packages/c4/c6/0417a92e6a3fc9b85f5a8380d9f9d43b69ba836a90e45f79f9ae74d41e53/eth_utils-5.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for poly_eip712_structs>=0.0.1 from https://files.pythonhosted.org/packages/5a/d7/ff1cfba1c3a3d5d6851d7bef5e4ad19710ed6d03e149dc183111d103acab/poly_eip712_structs-0.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for py-order-utils>=0.3.2 from https://files.pythonhosted.org/packages/29/68/b0a971b064b3236fce7307bd5c180409cccd9b207ec459274bdb4e401ec0/py_order_utils-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency

### PolyMarket API
https://docs.polymarket.com/quickstart/introduction/main

### NYC Mayoral Race 2025

#### Event: NYC Mayoral Election:
https://polymarket.com/event/new-york-city-mayoral-election?tid=1755656631578

#### Markets: 
- Mamdani
- Cuomo
- Adams
- Silwa

#### Get single event data using gamma api

In [1]:
import requests

r = requests.get("https://gamma-api.polymarket.com/events?id=23246")
nyc_mayoral_election_event = r.json()
nyc_mayoral_election_event

[{'id': '23246',
  'ticker': 'new-york-city-mayoral-election',
  'slug': 'new-york-city-mayoral-election',
  'title': 'New York City Mayoral Election',
  'description': 'The 2025 New York City mayoral election will be held on November 4, 2025.\n\nThis market will resolve according to the candidate wins the election.\n\nThe primary resolution source for this market will be a consensus of credible reporting, however if there is any ambiguity in the results this market will resolve according to official information from New York City.\n\n\n',
  'resolutionSource': '',
  'startDate': '2025-04-22T16:14:22.792059Z',
  'creationDate': '2025-04-22T16:14:22.792054Z',
  'endDate': '2025-11-04T12:00:00Z',
  'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/new-york-city-mayoral-election-al8KR4km2AQB.png',
  'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/new-york-city-mayoral-election-al8KR4km2AQB.png',
  'active': True,
  'closed': False,
  'archived': False,
  'new': 

In [2]:
for market in nyc_mayoral_election_event[0]['markets']:
    print(market['id'], market['question'], 'outcomePrices' in market and market['outcomePrices'])
    print('clobTokenIds' in market and market['clobTokenIds'])
    print("----------------------------")

538928 Will Eric Adams win the 2025 NYC mayoral election? ["0.0045", "0.9955"]
["12541509255877010177934715422358422748052378132651567836811989084820478865881", "557149893577824223387740663828120572195409127574902734254512226734855221907"]
----------------------------
538929 Will Andrew Cuomo win the 2025 NYC mayoral election? ["0.109", "0.891"]
["72685162394098505217895638060393901041260225434938300730127268362092284806692", "104468181147316868388088006861839293041095272602974154655578369735976654024471"]
----------------------------
538930 Will Curtis Sliwa win the 2025 NYC mayoral election? ["0.0045", "0.9955"]
["106645483208866512069057468273735379467092160028743318106715987586591689242591", "42541673615301895829890290486226257940966769125829226067368474110048691276042"]
----------------------------
538931 Will Jim Walden win the 2025 NYC mayoral election? ["0.0005", "0.9995"]
["10142872462975275995759595767341515402450823584364917228661247714894341308969", "33233690669868825436673

#### Statics Definiitons - START

In [3]:
"""
ASSET_ID_MAPPING = {
    "33945469250963963541781051637999677727672635213493648594066577298999471399137": "Mamdani_YES",
    "105832362350788616148612362642992403996714020918558917275151746177525518770551": "Mamdani_NO",
    "72685162394098505217895638060393901041260225434938300730127268362092284806692": "Cuomo_YES",
    "104468181147316868388088006861839293041095272602974154655578369735976654024471": "Cuomo_NO"
}
"""
ASSET_ID_MAPPING = {
    "29048360022556021389805670398008888482908398853670829781367251641936311260707": "Shai_YES",
    "114528627098181527180076013437205839368323282497361602702800503052375432480589": "Shai_NO",
    "73768610008619570600930429495180540710817177537162503586781057110775077618432": "Jokic_YES",
    "88794755386871079853762415286654635832909423950620116774027006364873482091563": "Jokic_NO",
    "89110596788673536475065853727140488937259064164063660201050220270400840228269": "Luka_YES",
    "101506943049053276934626391886226570064171431948041761918666910024462041911155": "Luka_NO"
}

In [17]:
SIDE_MAPPING = {
    "BUY": 0,
    "SELL": 1
}

#### Statics Definitions - END

#### Order Book - START

In [51]:
from collections import defaultdict

order_books = defaultdict(lambda: {"bids": {}, "asks": {}})

In [53]:
def update_order_book(asset_id, side, price, size):
    book_side = order_books[asset_id]["bids"] if side == "buy" else order_books[asset_id]["asks"]
    if size == 0:
        book_side.pop(price, None)
    else:
        book_side[price] = size

In [57]:
def print_top_of_book():
    # Print a snapshot (top of book)
    for aid, book in order_books.items():
        top_bid = max(book["bids"].items(), key=lambda x: x[0], default=(None, None))
        top_ask = min(book["asks"].items(), key=lambda x: x[0], default=(None, None))
        print(f"{ASSET_ID_MAPPING[aid]} | Best Bid: {top_bid} | Best Ask: {top_ask}")

In [60]:
def print_top_of_book_single_assest(book):
    if book["bids"]:
        best_bid = max(book["bids"].items())
    else:
        best_bid = (None, None)
    if book["asks"]:
        best_ask = min(book["asks"].items())
    else:
        best_ask = (None, None)
    print(f"Best Bid: {best_bid[0]} | Best Ask: {best_ask[0]}")

#### Order Book - END

#### Market Data Ingestion Bot - START

Polymarket Websocket Market Data: https://docs.polymarket.com/developers/CLOB/websocket/market-channel

In [4]:
import csv
import os
from datetime import datetime

In [5]:
# Get Mapping for Asset ID
def get_csv_filename(asset_id):
    mapped = ASSET_ID_MAPPING.get(asset_id, asset_id[:8])  # fallback
    return f"Polymarket_{mapped}.csv"

In [22]:
# Create new CSV file if one doesn't exist for Asset ID
def init_csv_if_needed(filename):
    if not os.path.exists(filename):
        with open(filename, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["timestamp", "event_type",
                             "price", "side", "size", "best_bid", "best_ask"])

In [23]:
# Write Row to Historical Data Store (CSV for now)
def write_row(asset_id, timestamp, event_type, price="", side="",
              size="", best_bid="", best_ask=""):
    filename = get_csv_filename(asset_id)
    init_csv_if_needed(filename)
    
    # Order data types from largest to smallest
    # Multiply floats by 1000 and store as shorts
    # Convert side to byte data type

    with open(filename, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            timestamp, event_type,
            price, side, size, best_bid, best_ask
        ])

In [24]:
# Write Book Message to CSV
def persist_book_event(message):
    asset_id = message["asset_id"]
    timestamp = message["timestamp"]
    bids = message.get("bids", [])
    asks = message.get("asks", [])

    # Best bid = highest price
    best_bid_price = bids[-1]["price"] if bids else ""
    best_bid_size = bids[-1]["size"] if bids else ""
    # Best ask = lowest price
    best_ask_price = asks[-1]["price"] if asks else ""
    best_ask_size = asks[-1]["size"] if asks else ""

    # Write BUY side
    write_row(asset_id, timestamp, "book",
              price=best_bid_price, side="BUY", size=best_bid_size,
              best_bid=best_bid_price, best_ask=best_ask_price)
    
    #Write SELL side
    write_row(asset_id, timestamp, "book",
              price=best_ask_price, side="SELL", size=best_ask_size,
              best_bid=best_bid_price, best_ask=best_ask_price)

In [32]:
# Write Price Change to CSV
def persist_price_change_event(message):
    timestamp = message["timestamp"]
    
    for pc in message["price_changes"]:
        write_row(
            asset_id=pc["asset_id"],
            timestamp=timestamp,
            event_type="price_change",
            price=pc.get("price", ""),
            side=pc.get("side", ""),
            size=pc.get("size", ""),
            best_bid=pc.get("best_bid", ""),
            best_ask=pc.get("best_ask", "")
        )
    

In [26]:
# Write tick size change event to CSV
def persist_tick_change_event(message):
    asset_id = message["asset_id"]
    timestamp = message["timestamp"]
    # No price/side/size/best bid/ask for this event
    write_row(asset_id, timestamp, "tick_size_change")

In [27]:
# Write last trade event to CSV
def persist_trade_event(message):
    asset_id = message["asset_id"]
    timestamp = message["timestamp"]

    write_row(
        asset_id=asset_id,
        timestamp=timestamp,
        event_type="last_trade_price",
        price=message.get("price", ""),
        side=message.get("side", ""),
        size=message.get("size", ""),
        best_bid="",  # Not provided
        best_ask=""
    )

#### Market Data Ingestion Bot - END

#### Get websocket data using Order Book API - START

In [35]:
import json
import threading
from websocket import WebSocketApp

# WebSocket endpoint for Polymarket CLOB service
WS_URL_BASE = "wss://ws-subscriptions-clob.polymarket.com"

# Your target tokens (clobTokenIds)
"""
ASSET_IDS = [
    "33945469250963963541781051637999677727672635213493648594066577298999471399137",  # Mamdani YES
    "105832362350788616148612362642992403996714020918558917275151746177525518770551",   # Mamdani NO
    "72685162394098505217895638060393901041260225434938300730127268362092284806692",  # Cuomo YES
    "104468181147316868388088006861839293041095272602974154655578369735976654024471"   # Cuomo NO
]
"""
ASSET_IDS = [
    "29048360022556021389805670398008888482908398853670829781367251641936311260707", # Shai YES
    "114528627098181527180076013437205839368323282497361602702800503052375432480589", # Shai NO
    "73768610008619570600930429495180540710817177537162503586781057110775077618432", # Jokic YES
    "88794755386871079853762415286654635832909423950620116774027006364873482091563", # Jokic NO
    "89110596788673536475065853727140488937259064164063660201050220270400840228269", # Luka YES
    "101506943049053276934626391886226570064171431948041761918666910024462041911155" # Luka NO
]

CHANNEL_TYPE = "market"  # use market for public price/book updates

class PolymarketWebSocket:
    def __init__(self, url_base, channel_type, asset_ids):
        self.url = f"{url_base}/ws/{channel_type}"
        self.channel_type = channel_type
        self.asset_ids = asset_ids
        self.ws = WebSocketApp(
            self.url,
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close,
        )

    def on_open(self, ws):
        print(f"Connected to {self.url}")
        # Subscribe to assets
        subscribe_payload = {
            "assets_ids": self.asset_ids,
            "type": self.channel_type
        }
        ws.send(json.dumps(subscribe_payload))
        # Start periodic ping to keep connection alive
        threading.Thread(target=self.ping, args=(ws,), daemon=True).start()

    def on_message(self, ws, message):
        msgs = json.loads(message)

        # Ensure msgs is always a list
        if isinstance(msgs, dict):
            msgs = [msgs]
            
        #print(msgs)
        #print(len(msgs))

        for msg in msgs:
            event_type = msg.get("event_type")
            #print(f"\nReceived message — event_type: {event_type}")
            #print(json.dumps(msg, indent=2))

            if event_type == "book":
                persist_book_event(msg)
                #self.handle_book(msg)
            elif event_type == "price_change":
                persist_price_change_event(msg)
                #self.handle_price_change(msg)
            elif event_type == "last_trade_price":
                persist_trade_event(msg)
                #self.handle_last_trade(msg)
            elif event_type == "tick_size_change":
                persist_tick_change_event(msg)
                #self.handle_tick_size_change(msg)
            else:
                print("Unrecognized message:", msg)
            
        # print_top_of_book()

    def on_error(self, ws, error):
        print("WebSocket error:", error)

    def on_close(self, ws, close_status_code, close_msg):
        print("WebSocket closed:", close_status_code, close_msg)

    def ping(self, ws):
        import time
        while True:
            ws.send("PING")
            time.sleep(10)

    def run(self):
        self.ws.run_forever()

    # Handler stubs — customize as needed
    def handle_book(self, msg):
        asset_id = msg["asset_id"]
        # UPDATE ORDER BOOK
        bids = {float(entry["price"]): float(entry["size"]) for entry in msg["bids"]}
        asks = {float(entry["price"]): float(entry["size"]) for entry in msg["asks"]}
        order_books[asset_id]["bids"] = bids
        order_books[asset_id]["asks"] = asks
        #print(f"[BOOK SNAPSHOT] {ASSET_ID_MAPPING[asset_id]}")
        #print_top_of_book_single_assest(order_books[asset_id])

    def handle_price_change(self, msg):
        #print("Price change update:", msg.get("changes"))
        asset_id = msg["asset_id"]
        # UPDATE ORDER BOOK
        for change in msg["changes"]:
            price = float(change["price"])
            size = float(change["size"])
            side = change["side"].lower()
            update_order_book(asset_id, side, price, size)
            
        #print(f"[PRICE UPDATE] {ASSET_ID_MAPPING[asset_id]}")
        #print_top_of_book_single_assest(order_books[asset_id])

    def handle_last_trade(self, msg):
        # UPDATE ORDER BOOK
        #print("Last trade price:", msg.get("price"), "size:", msg.get("size"))
        pass

    def handle_tick_size_change(self, msg):
        # UPDATE ORDER BOOK
        #print("Tick size change:", msg.get("old_tick_size"), "→", msg.get("new_tick_size"))
        pass

if __name__ == "__main__":
    client = PolymarketWebSocket(WS_URL_BASE, CHANNEL_TYPE, ASSET_IDS)
    client.run()


Connected to wss://ws-subscriptions-clob.polymarket.com/ws/market
WebSocket error: 


#### Get websocket data using Order Book API - END

### Kalshi

In [50]:
print_top_of_book()

Cuomo YES | Best Bid: (0.083, 12376.86) | Best Ask: (0.122, 12776.21)
Cuomo NO | Best Bid: (0.878, 12776.21) | Best Ask: (0.917, 12376.86)
Mamdani YES | Best Bid: (0.818, 32444.0) | Best Ask: (0.856, 30000.0)
Mamdani NO | Best Bid: (0.144, 30000.0) | Best Ask: (0.182, 32444.0)
